## This notebook is to calculate the fraction of sequences from each month that have been uploaded to GISAID and the data portal

### Notebook created: Apr 3, 2022
### Last updated: Apr 3, 2022

In [2]:
# import libraries
import pandas as pd

In [4]:
# import datasets
g_df = pd.read_csv('data/metadata_CANall_2022_03_08.csv')
g_df.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Virus_name,Type,Accession_ID,Collection_date,Location,Additional_location_information,Sequence_length,Host,Patient_age,Gender,...,Pangolin_version,Variant,AA_Substitutions,Submission_date,Is_reference?,Is_complete?,Is_high_coverage?,Is_low_coverage?,N-Content,GC-Content
0,hCoV-19/Canada/QC-L00254708/2020,betacoronavirus,EPI_ISL_1001069,2020-04-27,North_America_/_Canada_/_Quebec,NaN,29782,Human,28,Male,...,2022-02-28,NaN,"(M_D3G,NSP14_I150T,NSP12_P323L,Spike_D614G)",2021-02-15,NaN,True,True,NaN,NaN,0.380196
1,hCoV-19/Canada/QC-L00249423/2020,betacoronavirus,EPI_ISL_1001070,2020-04-14,North_America_/_Canada_/_Quebec,NaN,29782,Human,102,Female,...,2022-02-28,NaN,"(NSP13_S485L,NS3_T14I,NSP12_P323L,Spike_D614G)",2021-02-15,NaN,True,True,NaN,NaN,0.379961
2,hCoV-19/Canada/QC-L00249435/2020,betacoronavirus,EPI_ISL_1001071,2020-04-14,North_America_/_Canada_/_Quebec,NaN,29782,Human,92,Female,...,2022-02-28,NaN,"(NSP13_S485L,NS3_T14I,NSP12_P323L,Spike_D614G)",2021-02-15,NaN,True,True,NaN,NaN,0.379961
3,hCoV-19/Canada/QC-L00249382/2020,betacoronavirus,EPI_ISL_1001072,2020-04-14,North_America_/_Canada_/_Quebec,NaN,29782,Human,93,Female,...,2022-02-28,NaN,"(NSP13_S485L,NS3_T14I,NSP12_P323L,Spike_D614G)",2021-02-15,NaN,True,True,NaN,NaN,0.379940
4,hCoV-19/Canada/QC-L00249363/2020,betacoronavirus,EPI_ISL_1001073,2020-04-14,North_America_/_Canada_/_Quebec,NaN,29782,Human,96,Female,...,2022-02-28,NaN,"(NSP13_S485L,NS3_T14I,NSP12_P323L,Spike_D614G)",2021-02-15,NaN,True,True,NaN,NaN,0.379940


In [5]:
g_df.columns

Index(['Virus_name', 'Type', 'Accession_ID', 'Collection_date', 'Location',
       'Additional_location_information', 'Sequence_length', 'Host',
       'Patient_age', 'Gender', 'Clade', 'Pango_lineage', 'Pangolin_version',
       'Variant', 'AA_Substitutions', 'Submission_date', 'Is_reference?',
       'Is_complete?', 'Is_high_coverage?', 'Is_low_coverage?', 'N-Content',
       'GC-Content'],
      dtype='object')

In [6]:
g_data = g_df[['Accession_ID', 'Collection_date', 'Location', 'Submission_date']]
g_data.head()

,Accession_ID,Collection_date,Location,Submission_date
0,EPI_ISL_1001069,2020-04-27,North_America_/_Canada_/_Quebec,2021-02-15
1,EPI_ISL_1001070,2020-04-14,North_America_/_Canada_/_Quebec,2021-02-15
2,EPI_ISL_1001071,2020-04-14,North_America_/_Canada_/_Quebec,2021-02-15
3,EPI_ISL_1001072,2020-04-14,North_America_/_Canada_/_Quebec,2021-02-15
4,EPI_ISL_1001073,2020-04-14,North_America_/_Canada_/_Quebec,2021-02-15


In [10]:
g_data.Location.unique()

array(['North_America_/_Canada_/_Quebec',
       'North_America_/_Canada_/_Newfoundland_and_Labrador',
       'North_America_/_Canada_/_Saskatchewan',
       'North_America_/_Canada_/_Ontario',
       'North_America_/_Canada_/_British_Columbia',
       'North_America_/_Canada_/_Alberta',
       'North_America_/_Canada_/_Manitoba',
       'North_America_/_Canada_/_Nova_Scotia',
       'North_America_/_Canada_/_New_Brunswick', 'North_America_/_Canada',
       'North_America_/_Canada_/_Alberta_/_Calgary',
       'Asia_/_China_/_Hubei_/_Wuhan',
       'North_America_/_Canada_/_Ontario_/_Toronto',
       'North_America_/_Canada_/_Ontario_/_Ottawa',
       'North_America_/_Canada_/_Newfoundland',
       'North_America_/_Canada_/_Ontario_/_Brampton',
       'North_America_/_Canada_/_Ontario_/_Regional_Municipality_of_Durham'],
      dtype=object)

In [11]:
import re
g_data.Location = g_data.Location.str.replace('North_America_/_Canada_/_', '')
g_data.head()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Accession_ID,Collection_date,Location,Submission_date
0,EPI_ISL_1001069,2020-04-27,Quebec,2021-02-15
1,EPI_ISL_1001070,2020-04-14,Quebec,2021-02-15
2,EPI_ISL_1001071,2020-04-14,Quebec,2021-02-15
3,EPI_ISL_1001072,2020-04-14,Quebec,2021-02-15
4,EPI_ISL_1001073,2020-04-14,Quebec,2021-02-15


In [12]:
g_data.Location.unique()

array(['Quebec', 'Newfoundland_and_Labrador', 'Saskatchewan', 'Ontario',
       'British_Columbia', 'Alberta', 'Manitoba', 'Nova_Scotia',
       'New_Brunswick', 'North_America_/_Canada', 'Alberta_/_Calgary',
       'Asia_/_China_/_Hubei_/_Wuhan', 'Ontario_/_Toronto',
       'Ontario_/_Ottawa', 'Newfoundland', 'Ontario_/_Brampton',
       'Ontario_/_Regional_Municipality_of_Durham'], dtype=object)